## Úkol 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shippy/czechitas-ai-data/blob/main/notebooks/assignment-03.ipynb)


### Nastavte si API klíč

1. Navštivte [Google AI Studio](https://aistudio.google.com/app/apikey) a vytvořte si API klíč.
2. V levé liště klikněte na tlačítko "Secrets" a přidejte předtím vytvořený API klíč do proměnné `GOOGLE_API_KEY` - buď "napřímo" přes "+ Add new secret", nebo přes "Gemini API keys" > "Import key from Google AI Studio".

(Pokud jste otevřeli tento notebook lokálně, např. ve VSCode, můžete si API klíč nastavit v souboru `.env`.)

### Setup a test

Spusťte následující dvě buňky. Pokud vám AI řekne, jak se má, vše funguje správně.

In [2]:
try:
    from google.colab import userdata
    _secret = userdata.get("GOOGLE_API_KEY")
    %pip install instructor google-generativeai python-dotenv rich
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("GOOGLE_API_KEY")

In [ ]:
import instructor
import google.generativeai as genai
import os
from rich import print

try:
    if _secret:
        genai.configure(api_key=_secret)
    else:
        raise ValueError("API klíč nebyl nastaven!")
except NameError:
    print("Nastavte si API klíč v proměnné GOOGLE_API_KEY a znovu spusťte *celý* notebook včetně předchozí buňky")

_client = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash-latest",
)
# _client.generate_content pro "normální" použití
client = instructor.from_gemini(
    client=_client,
    mode=instructor.Mode.GEMINI_JSON,
)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you? Respond with an emotion."},
    ],
    response_model=str,
)
print(test)


/Users/simon/Documents/czechitas-ai-data/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello! I am doing well, thank you for asking. 😊


### Úkol, část 1

Vytáhněte ze nestrukturovaných odpovědí níže definované datové struktury.

In [7]:
from pydantic import BaseModel, Field
from enum import Enum

class Occupation(str, Enum):
    GARDENER = "gardener"
    TEACHER = "teacher"
    DOCTOR = "doctor"
    ENGINEER = "engineer"
    LAWYER = "lawyer"
    SCIENTIST = "scientist"
    ARTIST = "artist"
    WRITER = "writer"
    OTHER = "other"

# Alternativně jsme mohli použít i typing.Literal následovně:

# from typing import Literal
# Occupation = Literal["gardener", "teacher", "doctor", "engineer", "lawyer", "scientist", "artist", "writer", "other"]


class Citizen(BaseModel):
    name: str = Field(..., description="Full name of the citizen")
    age: int
    occupation: Occupation
    interests: list[str]

In [8]:
census_responses = [
    "Jsem osmadvacetiletý Honza Pálka, pracuju v Semilhotce jako čistič kanálů. Rád hraju šachy a na bicí.",
    "Jsem Jana Nováková, je mi 30, dojíždím do Semilhotské školy učit. Ráda čtu knihy a jezdím na hory.",
    "Jsem čtyřicetiletý Petr Válek, pracuju v Semilhotce jako zedník. Rád cestuji a hraju fotbal.",
    "Jsem padesátiletý Vladimír Dvořák, pracuju v Semilhotce jako policista. Rád cestuji a hraju kulečník.",
    "Jsem šedesátiletý Jaroslav Bílek, pracuju v Semilhotce jako zahradník. Rád cestuji a hraju kulečník.",
    "Jsem sedmdesátiletý Karel Dvořák, pracuju v Semilhotce jako zedník. Rád cestuji a hraju kulečník.",
    "Jsem osmdesátiletý Josef Novák, pracuju v Semilhotce jako zahradník. Rád cestuji a hraju kulečník.",
    "Jsem devadesátiletý Igor Hnízdo, jsem tu penzista. Rád cestuji a hraju kulečník.",
]


- Tip 1: můžete použít i prompt engineering (tj. první message s `"role": "system"`), aby AI lépe poznala, co chcete.
- Tip 2: `client.chat.completions.create` umí vracet i `response_model=Citizen` nebo i `response_model=list[Citizen]`

Dávalo by smysl na základě dat přidefinovat do třídy `Citizen` nějaký atribut, abyste zachytili více informací?

In [ ]:
# Zde řešte úkol 1.

### Úkol, část 2

Vygenerujte dataset nebo sadu datasetů, který popíše fiktivní městečko Semilhotka. Začněte např. s následujícími datovými strukturami:

In [ ]:

# A vygenerujeme nějaké obyvatele
citizens = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Generate 10 citizens of Semilhotka."},
    ],
    response_model=list[Citizen],
)
print(citizens)

Ideálně bychom chtěli, aby výčet kulminoval objektem Semilhotka:

```python
from pydantic import BaseModel

class Semilhotka(BaseModel):
    citizens: list[Citizen]
    # a další atributy

my_semilhotka = Semilhotka(
    citizens=citizens,
    # a další proměnné, které jste si vygenerovali
)
```

